# **NB01 - Data Collection**

**OBJECTIVE:**
Collect weather data from the OpenMeteo API and save it to a JSON file. The weather data will be extracted from ten different cities, including London, and will include the following:
- The Annual precipitation 2023
- The number of days of rainfall in 2023
- The average annual precipitation in mm over a 20 year period 
- The average annual number of days of rainfall over a 20 year period

**AUTHOR:** 
@nadiabegic on GitHub

**LAST EDITED:**
1-Nov-2024

-----------------------
**Imports**:

In [2]:
import requests
import os
import json
from datetime import datetime

# 1. Collect the number of days of rainfall in 2023

In [ ]:
# tweak this function a bit

def get_historical_data(country_code, city_name, start_date, end_date, world_cities):
    """
    Retrieves the historical temperatures for a given country code and city name.

    Parameters:
        country_code (str): The country code of the location.
        city_name (str): The name of the city.
        start_date (str): The start date of the historical data in the format 'YYYY-MM-DD'.
        end_date (str): The end date of the historical data in the format 'YYYY-MM-DD'.
        world_cities (dict): A dictionary containing world cities data.

    Returns:
        list: A list of historical temperatures (in Celsius) for the given date range, in hourly intervals.
    """

    latitude, longitude = get_lat_long(country_code, city_name, world_cities)

    base_historical_url = "https://archive-api.open-meteo.com/v1/era5?"
    params_lat_long     = "latitude=" + str(latitude) + "&longitude="  + str(longitude)
    params_date         = "&start_date=" + str(start_date) + "&end_date=" + str(end_date)
    params_others       = "&hourly=temperature_2m"

    final_url = base_historical_url + params_lat_long + params_date + params_others

    response = requests.get(final_url)

    historical_data = response.json()
    historical_temperatures = historical_data['hourly']['temperature_2m']
    return historical_temperatures

# Demonstrate the this function works as intended
london_historical = get_historical_data('GB', 'London', '2023-10-13', '2023-10-19', world_cities)
print(f"The function returned a list of {len(london_historical)} elements.")
print(f"Head of the list: {london_historical[0:10]}")
print(f"Tail of the list: {london_historical[-10:]}")